# Structured Output Parser

This output parser can be used when you want to return multiple fields. While the Pydantic/JSON parser is more powerful, we initially experimented with data structures having text fields only.

In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

Here we define the response schema we want to receive.

In [2]:
response_schemas = [
    ResponseSchema(name="answer", description="Answer to the user's question"),
    ResponseSchema(name="source", description="Website used to answer the user's question")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

We now get a string that contains instructions for how the response should be formatted.

In [3]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // Answer to the user's question
	"source": string  // Website used to answer the user's question
}
```


We can now use this to format a prompt to send to the language model, and then parse the returned result.

In [4]:
template = """Answer the users question as best as possible.

{format_instructions}

Question: {question}"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

prompt_value = prompt_template.format_prompt(
    question="What's the capital of France?",
)
print(prompt_value.to_string())

Answer the users question as best as possible.

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // Answer to the user's question
	"source": string  // Website used to answer the user's question
}
```

Question: What's the capital of France?


In [5]:
llm = OpenAI(temperature=0)
raw_output = llm(prompt_value.to_string())
raw_output

'\n\n```json\n{\n\t"answer": "Paris",\n\t"source": "https://en.wikipedia.org/wiki/France"\n}\n```'

In [6]:
parsed_output = output_parser.parse(raw_output)
parsed_output

{'answer': 'Paris', 'source': 'https://en.wikipedia.org/wiki/France'}